In [ ]:
from arcgis.gis import GIS
from arcgis.raster import export_image

# Initialize the GIS object
gis = GIS("https://your-arcgis-server-url.com", username="your-username", password="your-password")

# Define the MapServer URL
map_service_url = "https://your-arcgis-server-url.com/arcgis/rest/services/YourMapService/MapServer"

# Create a MapService object
map_service = gis.content.get(map_service_url)

# Set the output directory for downloaded tiles
output_directory = "path/to/output/directory"

# Define the extent and zoom levels you want to download
extent = {"xmin": -180, "ymin": -90, "xmax": 180, "ymax": 90, "spatialReference": {"wkid": 4326}}
zoom_levels = [0, 1, 2, 3]  # Adjust this list as needed

for zoom_level in zoom_levels:
    # Export tiles at the specified zoom level
    export_image(map_service, bbox=extent, export_format="png", size=[256, 256], level_of_detail=zoom_level,
                 out_folder=output_directory)

print("Tile download completed.")


In [3]:
import os
import requests
from bs4 import BeautifulSoup

def download_folder(url, local_path):
    # Send an HTTP GET request to the folder URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the folder page
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Create the local folder if it doesn't exist
        if not os.path.exists(local_path):
            os.makedirs(local_path)
        
        # Find all links (anchor tags) in the HTML content
        links = soup.find_all('a')
        
        # Iterate through the links and download files/subfolders
        for link in links:
            link_url = link.get('href')
            
            # Ensure the link is not a parent directory link or a link to a different domain
            if link_url and not link_url.startswith('..') and not link_url.startswith('http'):
                full_url = url + link_url
                file_name = os.path.basename(full_url)
                file_path = os.path.join(local_path, file_name)
                
                # If the link points to a subfolder, recursively download its contents
                if link_url.endswith('/'):
                    download_folder(full_url, file_path)
                else:
                    # Otherwise, download the file
                    file_response = requests.get(full_url)
                    
                    if file_response.status_code == 200:
                        with open(file_path, 'wb') as file:
                            file.write(file_response.content)
                        print(f"Downloaded: {file_path}")
                    else:
                        print(f"Failed to download: {full_url}")
    else:
        print(f"Failed to access the folder: {url}")

# Example usage:
folder_url = 'https://maps.simcoe.ca/arcgis/rest/services/Public/Ortho_1989_Cache/MapServer/tile/9/94842'
local_folder = 'downloaded_folder'
download_folder(folder_url, local_folder)


Failed to access the folder: https://maps.simcoe.ca/arcgis/rest/services/Public/Ortho_1989_Cache/MapServer/tile/9/94842
